## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [ ]:
NAME = "THEODOROS"
AEM = "IOANNIDIS"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [ ]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [ ]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7f0ee76d1750>)

In [ ]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [ ]:
# BEGIN CODE HERE
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection

cls1 = DecisionTreeClassifier(random_state= RANDOM_STATE) # Classifier #1 
cls2 = KNeighborsClassifier(n_neighbors= 3) # Classifier #2 
cls3 = LogisticRegression(random_state= RANDOM_STATE) # Classifier #1
soft_vcls = VotingClassifier([('DecisionTree', cls1), ('KNeighbors', cls2), ('Logistic', cls3)], voting = 'soft') # Voting Classifier
hard_vcls = VotingClassifier([('DecisionTree', cls1), ('KNeighbors', cls2), ('Logistic', cls3)], voting = 'hard') # Voting Classifier

svlcs_scores = model_selection.cross_validate(soft_vcls, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
s_avg_fmeasure = svlcs_scores['test_f1_weighted'].mean() # The average f-measure
s_avg_accuracy = svlcs_scores['test_balanced_accuracy'].mean() # The average accuracy

hvlcs_scores = model_selection.cross_validate(hard_vcls, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
h_avg_fmeasure = hvlcs_scores['test_f1_weighted'].mean() # The average f-measure
h_avg_accuracy = hvlcs_scores['test_balanced_accuracy'].mean() # The average accuracy
#END CODE HERE

In [ ]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('DecisionTree',
                              DecisionTreeClassifier(random_state=42)),
                             ('KNeighbors',
                              KNeighborsClassifier(n_neighbors=3)),
                             ('Logistic', LogisticRegression(random_state=42))],
                 voting='soft')
F1 Weighted-Score: 0.8286 & Balanced Accuracy: 0.8212


You should achive above 82% (Soft Voting Classifier)

In [ ]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('DecisionTree',
                              DecisionTreeClassifier(random_state=42)),
                             ('KNeighbors',
                              KNeighborsClassifier(n_neighbors=3)),
                             ('Logistic', LogisticRegression(random_state=42))])
F1 Weighted-Score: 0.8296 & Balanced Accuracy: 0.8224


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




##### Tests: 
    
* final_estimator = KNearestNeighbors(n_neighbors = 3)
   
    * MLPClassifier με hidden_layer_sizes = (100,), SVC with default parameters and the classifier achieved 83-84% weighted f1 and balanced accuracy scores.

    * Replaced SVC with XGBoost and the classifier performed worse 81-82%. Also it was significantly slower.

I decided to add another complex estimator, hoping to achieve better results. Also, this time I used LogisticRegression as the final estimator.

* final_estimator= LogisticRegression()

    * MLPClassifier(hidden_layer_sizes = (100,)), RandomForestClassifier, SVC with default parameters.

The results were F1 Weighted Score: 0.8657 & Balanced Accuracy: 0.8593. Increasing the number of complex estimators definitely had a negative impact on the time. With that being said

In [ ]:
# BEGIN CODE HERE
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn import model_selection

cls1 = MLPClassifier(hidden_layer_sizes= (100,), activation= 'relu', random_state= RANDOM_STATE) # Classifier #1 
cls2 = RandomForestClassifier(random_state=RANDOM_STATE) # Classifier #2 
cls3 = SVC(random_state=RANDOM_STATE) # Classifier #3
final_cls = LogisticRegression(random_state= RANDOM_STATE)
scls = StackingClassifier([('mlp', cls1), ('rf', cls2), ('SVC', cls3)], final_estimator=  final_cls) # Stacking Classifier
scores = model_selection.cross_validate(scls, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
avg_fmeasure = scores['test_f1_weighted'].mean() # The average f-measure
avg_accuracy = scores['test_balanced_accuracy'].mean() # The average accuracy
#END CODE HERE

In [ ]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('mlp', MLPClassifier(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('SVC', SVC(random_state=42))],
                   final_estimator=LogisticRegression(random_state=42))
F1 Weighted Score: 0.8657 & Balanced Accuracy: 0.8593


You should achieve above 85% in both

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [ ]:
# BEGIN CODE HERE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
ens1 = RandomForestClassifier(n_estimators=200 ,random_state= RANDOM_STATE)
ens2 = ExtraTreesClassifier(random_state= RANDOM_STATE)
ens3 = GradientBoostingClassifier(random_state= RANDOM_STATE)
tree = DecisionTreeClassifier(random_state= RANDOM_STATE)

score1 = model_selection.cross_validate(ens1, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
score2 = model_selection.cross_validate(ens2, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
score3 = model_selection.cross_validate(ens3, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
score4 = model_selection.cross_validate(tree, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])

f_measures = dict({"Random Forest": score1['test_f1_weighted'].mean() , "Extra Trees": score2['test_f1_weighted'].mean(),
                   "Grad Boost": score3['test_f1_weighted'].mean(), "Decision Tree": score4['test_f1_weighted'].mean() })

accuracies = dict({"Random Forest": score1['test_balanced_accuracy'].mean() , "Extra Trees": score2['test_balanced_accuracy'].mean(),
                   "Grad Boost": score3['test_balanced_accuracy'].mean(), "Decision Tree": score4['test_balanced_accuracy'].mean()})
# Example f_measures = {'Simple Decision': 0.8551, 'Ensemble with random ...': 0.92, ...}

#END CODE HERE

In [ ]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,round(score,4)))

RandomForestClassifier(random_state=42)
ExtraTreesClassifier(random_state=42)
GradientBoostingClassifier(random_state=42)
DecisionTreeClassifier(random_state=42)
Classifier:Random Forest -  F1 Weighted:0.8048
Classifier:Extra Trees -  F1 Weighted:0.8083
Classifier:Grad Boost -  F1 Weighted:0.8197
Classifier:Decision Tree -  F1 Weighted:0.7046
Classifier:Random Forest -  BalancedAccuracy:0.7901
Classifier:Extra Trees -  BalancedAccuracy:0.7939
Classifier:Grad Boost -  BalancedAccuracy:0.8089
Classifier:Decision Tree -  BalancedAccuracy:0.6957


**2.2** Describe your classifiers and your results.


The classifiers I tested were: RandomForestClassifier, Extra-Trees Classifiers, Gradient Boosting Classifier and a Simple Decision Tree Classifier. The best Weighted F1 score and Balanced Accuracy were produced from the Gradient Boosting Classifier (F1 Weighted: 0.8197, BalancedAccuracy: 0.8089). The second best model was the ExtraTreesClassifier (F1 Weighted: 0.8083, BalancedAccuracy: 0.7939), the third best was Random Forest with 200 estimators (F1 Weighted: 0.8048, BalancedAccuracy: 0.7901) The Gradient Boosting Classifier was the slowest of all. 
As expected, the simple decision tree had the worst performance (F1 Weighted: 0.7046, BalancedAccuracy: 0.6957).

(I chose to set the n_estimators in RandomForestClassifier to 200 in order to, hopefully, achieve better results).

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

Bagging produces heterogenous ensembles, and therefore, the models can be created and trained in parallel (by passing the arguement n_jobs = -1 in the Bagging Classifier). 
Boosting produces homogenous ensembles. This means that the models have to be created and trained sequentially. Unfortunately, this solution cannot be applied to boosting classifiers.

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [13]:
# BEGIN CODE HERE
#for the stacking classifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn import model_selection

cls1 = MLPClassifier(hidden_layer_sizes= (100,), activation= 'relu', random_state= RANDOM_STATE) # Classifier #1 
cls2 = RandomForestClassifier(random_state=RANDOM_STATE) # Classifier #2 
cls3 = SVC(random_state=RANDOM_STATE) # Classifier #3
final_cls = LogisticRegression(random_state= RANDOM_STATE)
best_cls = StackingClassifier([('MLP', cls1), ('RF', cls2), ('SVC', cls3)], final_estimator=  final_cls)

scores = model_selection.cross_validate(best_cls, X, y, cv= 10, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])

best_fmeasure = round(scores['test_f1_weighted'].mean(), 4)
best_accuracy = round(scores['test_balanced_accuracy'].mean(),4)

#END CODE HERE

In [14]:
print("Classifier:")
print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
StackingClassifier(estimators=[('MLP', MLPClassifier(random_state=42)),
                               ('RF', RandomForestClassifier(random_state=42)),
                               ('SVC', SVC(random_state=42))],
                   final_estimator=LogisticRegression(random_state=42))
F1 Weighted-Score:0.8657 & Balanced Accuracy:0.8593


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

I chose the stacking classifier from 1.2, because it that had the best weighted f1 and balanced accuracy scores out of all the other models that I tested. The results were: 

F1 Weighted-Score: 0.8657

Balanced Accuracy: 0.8593

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [ ]:
# BEGIN CODE HERE
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn import model_selection

cls1 = MLPClassifier(hidden_layer_sizes= (100,), activation= 'relu', random_state= RANDOM_STATE) # Classifier #1 
cls2 = RandomForestClassifier(random_state=RANDOM_STATE) # Classifier #2 
cls3 = SVC(random_state=RANDOM_STATE) # Classifier #3
final_cls = LogisticRegression(random_state= RANDOM_STATE)
cls = StackingClassifier([('mlp', cls1), ('rf', cls2), ('SVC', cls3)], final_estimator=  final_cls)
cls.fit(X, y)
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

I chose to use the Stacking Classifier with 3 complex estimators (MLP,RandomForestClassifier and SVC) and Logistic Regression as the final classifier that outperformed all the other models that I tested. It achieved F1 Weighted-Score: 0.8657
and Balanced Accuracy: 0.8593.

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!